In [ ]:
import glob
from bs4 import BeautifulSoup
import requests
import string
import pandas as pd
import requests
import os

In [ ]:
path = "lexile/raw/lists/"
files = [f for f in glob.glob(path + "*.html", recursive=True)]
base_url = 'https://www.commonsensemedia.org/book-reviews'

In [ ]:
def get_titles(soup):
    for s in soup.findAll(class_="views-field views-field-field-reference-review-ent-prod result-title"):
        titles.append(s.get_text().strip())
    return titles

In [ ]:
def edit_titles(titles):
    stripper = str.maketrans("", "", string.punctuation)
    titles_edited = [t.translate(stripper).replace(" ", "-").lower() for t in titles]
    return titles_edited

In [ ]:
def make_urls(title_edited):
    urls = [base_url + "/" + t for t in title_edited]
    return urls

In [ ]:
titles = []

for file in files:
    with open(file, 'r') as f: 
        soup = BeautifulSoup(f.read(), 'html.parser')
        get_titles(soup)
        
titles_edited = edit_titles(titles)

urls = make_urls(titles_edited)

In [ ]:
df = pd.DataFrame()
df['title'] = titles
df['title_edited'] = titles_edited
df['url'] = urls
df.loc[:, "title_edited"] = df.loc[:, 'title_edited'].str.replace("--", "-")
df.loc[:, "url"] = df.loc[:, "url"].str.replace("--", "-")
df.head(5)

In [ ]:
df.shape

# Scrape Each Book's Review Page

In [ ]:
!mkdir lexile/raw/books

In [ ]:
for i, j in df.iterrows():
    if not os.path.isfile(path + j.title_edited + ".html"):
        page = requests.get(j.url)
        soup = BeautifulSoup(page.text, 'html.parser')
        with open("lexile/raw/books/" + j.title_edited + ".html", 'wb') as f:
            try:
                f.write(soup.encode('utf-8'))
            except:
                with open("lexile/raw/book-errors.txt", 'a+') as f:
                    f.write(j.title + "\n")
    else: 
        continue